In [33]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)


from functions import Separate_data, clean_table_mp  
from functions import read_and_check_input_values 
from functions import create_matrix_A_and_C,format_constraints_elements
from functions import format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import optimize_with_correction
from functions import  construct_result_dataframe,export_result,save_errors,remove_old_recipes

In [42]:
def create_optimal_recipe(recette,table_mp, mp_constraints, elmt_quality_constraints,dossier_data):
    df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]

    df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)
    # Suppression des matières premières indisponibles dans
    table_mp = clean_table_mp(table_mp, df_mp_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)
    
    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)

    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'
    coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    # res, erreurs = optimize_with_correction(table_mp, C, constraints, bounds, method, coefficients)
    res, erreurs = solve_linear_program(C, constraints, bounds,method)
    
    # Gestion du recettes
    if not erreurs: 
        # Construire le DataFrame résultats
        df_res,contraints_res = construct_result_dataframe(res, df_mp_dispo, table_mp,constraints)
        # Écrire le DataFrame résultats dans le fichier Excel
        export_result(df_res, dossier_data, new_sheet_name= recette)
        print("Le problème pour la recette" ,recette,"admet une solution.")

    else :
        # Sauvegarder les erreurs dans un fichier texte
        save_errors(erreurs, dossier_data,recette)
        print("Les erreurs du problème " ,recette," ont été enregistrées dans un fichier")
    return 

In [43]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    # Suppression du vieux resultats
    remove_old_recipes(dossier_data)
    # Resolutions du nouveau probleme
    Recettes = [col for col in pd.read_excel(chemin_fichier, engine='calamine', sheet_name=2).columns if 'Unnamed' not in col]
    for recette in tqdm(Recettes, desc="Processing recipes", unit="recipe"):
        table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_input_values(chemin_fichier)
        if errors :
            save_errors(errors, dossier_data,recette)
            break;
        create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_data)


Processing recipes:  25%|██▌       | 1/4 [00:00<00:01,  2.84recipe/s]

Le problème pour la recette GS 400-15 admet une solution.


Processing recipes:  50%|█████     | 2/4 [00:00<00:00,  2.39recipe/s]

Le problème pour la recette GS 450-10 admet une solution.


Processing recipes:  75%|███████▌  | 3/4 [00:01<00:00,  2.76recipe/s]

Les erreurs du problème  GS 500-7  ont été enregistrées dans un fichier


Processing recipes: 100%|██████████| 4/4 [00:01<00:00,  2.84recipe/s]

Les erreurs du problème  GS 600-3  ont été enregistrées dans un fichier


## Autres


In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)


from functions import Separate_data, clean_table_mp  
from functions import read_and_check_input_values 
from functions import create_matrix_A_and_C,format_constraints_elements
from functions import format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import optimize_with_correction
from functions import  construct_result_dataframe,export_result,save_errors,remove_old_recipes

In [9]:
import numpy as np

chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
# On recupere le chemin du dossier data
dossier_data = os.path.dirname(chemin_fichier)
# Suppression du vieux resultats
remove_old_recipes(dossier_data)
# Resolutions du nouveau probleme
recette = 'GS 400-15'

table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_input_values(chemin_fichier)

df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]

df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)
# Suppression des matières premières indisponibles dans
table_mp = clean_table_mp(table_mp, df_mp_indispo)

# Construction de la matrice A et du vecteur C
A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)

# Initialisation des listes pour les contraintes
constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

# Mettre  les contraintes concernant les éléments
constraints = format_constraints_elements(df_contraints_element, A,constraints)

# Mettre  les contraintes concernant la qualité
constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

# Mettre les contraintes concernant les matières premières disponibles
constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)

# Résoudre le problème d'optimisation linéaire
method = 'simplex' #'interior-point' 'simplex'
coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]



A_ub= np.array(list(constraints['A_sup'].values()))
A_eq = np.array(list(constraints['A_eq'].values()))
b_ub = np.array(list(constraints['b_sup'].values()))
b_eq = np.array(list(constraints['b_eq'].values()))


In [10]:
import sympy as sp

# Define the symbols
x = sp.symbols('x0:24')  # Assuming 3 variables x0, x1, x2

# Définir les dimensions de A et b
selected_rows_indices = [0, 2, 4]  
A_np = A_ub[selected_rows_indices]#prendre uniquement A_ub.shape = (3,24)
b_np = b_ub[selected_rows_indices]

C_np = C #C[selected_rows_indices]

# Convertir en matrices SymPy
A = sp.Matrix(A_np)
b = sp.Matrix(b_np)

C = sp.Matrix(C_np)


# List to store inequalities
inequalities = []

# Construct inequalities
for i in range(A.shape[0]):
    inequality = sum(A[i, j] * x[j] for j in range(A.shape[1])) - b[i]  # Define the inequality
    inequalities.append(inequality)

# Solve the inequalities
solution = sp.solve(inequalities, *x)


Cout = sum(C[j] * x[j] for j in range(C.shape[0]))
Cout
solution
# Donner l'ensemble des solutions x, qui minimise C sous contrainte solution < 0

{x0: 16.7612293257378*x3 + 2.16784604705713*x4 - 4.27027932693458*x5 + 0.939275712678618*x6 + 0.120692213791618*x7 - 0.252567079154599*x8 - 0.0687426697623208*x9 + 0.216352713085521,
 x1: -64.6500203451495*x3 + 0.964768903037411*x4 - 1.10534000335097*x5 - 0.0829124679863089*x6 - 0.374427343880897*x7 - 0.947533449819287*x8 - 0.417865434787812*x9 - 0.198089949496158,
 x2: -17.5609957155509*x3 - 3.50260427487494*x4 + 4.45630101247038*x5 - 2.03430431556524*x6 - 0.143738241700376*x7 + 0.261892338256062*x8 + 0.0550563680317863*x9 + 0.822891883482132}

In [11]:
inequalities # les inconnus sont 

[3.65*x0 + 0.1*x1 + 3.5*x2 + 6.75*x3 + 4.25*x4 + 0.1*x5 + 3.7*x6 + 0.1*x7 + 0.1*x8 + 0.1*x9 - 3.65,
 2.64*x0 + 0.175*x1 + 1.9*x2 + 0.43*x3 + 0.763*x4 + 3.0*x5 + 1.4*x6 + 0.02*x7 + 0.335*x8 + 0.15*x9 - 2.1,
 0.22*x0 + 1.1*x1 + 0.45*x2 + 75.33*x3 + 0.038*x4 + 0.15*x5 + 0.8*x6 + 0.45*x7 + 0.98*x8 + 0.45*x9 - 0.2]

In [12]:
Cout

530*x0 + 430*x1 + 430*x2 + 1380*x3 + 530*x4 + 493.9*x5 + 435*x6 + 453*x7 + 423.9*x8 + 413.8*x9